In [327]:
import sys

import folium
from folium.plugins import MarkerCluster, HeatMap, MeasureControl
import branca
import numpy as np
import pandas as pd
import geopandas as gpd
from pprint import pprint as pp
from pymongo import MongoClient
import json
from scipy import stats


# set property type of interest
property_types = ['House', 'Condo', 'Duplex']
prop = property_types[0]

#connect to database
client = MongoClient('mongodb://localhost:27017/')
properties_db = client['properties']

#collections
liquidity_db = properties_db['LiquidityPremium']
listing_db = properties_db['ListingDetails']

#create pandas dataframe
listing_df = pd.DataFrame(list(listing_db.find({})))
market_df = pd.DataFrame(list(liquidity_db.find({})))

print(f'Number of listings: {len(listing_df)}')
print(f'Number of market regions: {market_df.name.nunique()}')

# Number of listings per month
listing_df['month'] = listing_df['last_modified'].dt.month
month_name = listing_df['last_modified'].dt.month_name()
month_name.value_counts()

# copy geopoints into columns
unpack_lat = lambda x: x['lat']
unpack_lng = lambda x: x['lng']
listing_df['lat'] = listing_df.geo_points.transform(unpack_lat)
listing_df['lng'] = listing_df.geo_points.transform(unpack_lng)

#drop erroneous columns
drop_cols = ['rooms 16', 'rooms 17', 'rooms 12']

notn = listing_df[drop_cols].notnull()
indexes = []
for idx, col in enumerate(drop_cols):
    indexes.append(notn[notn[col] == True].index[0])

# Drop erroneous noise
listing_df = listing_df.drop(indexes)
listing_df = listing_df.drop(columns=drop_cols)

#Replace Unknown, To Be Built, Under construction
listing_df['Year built'] = listing_df['Year built'].replace('Unknown age', np.nan)
func = lambda x: (2020 - x) if isinstance(x, int) else 0
listing_df['age'] = listing_df['Year built'].transform(func)

#Replace Condominium with condo
listing_df['type'] = listing_df['type'].replace('Condominium', 'Condo')


"""
# Add address processing
def split_geocode(val):
    for value in val:
        query = " ".join(value['list_address'].split(',')[2:]).strip()
        return gpd.tools.geocode(query, provider="nominatim")['address']

listing_df['geo_code'] = listing_df['list_address'].apply(split_geocode)
"""


# Create geodataframe
listing_geo = gpd.GeoDataFrame(listing_df, geometry=gpd.points_from_xy(listing_df.lng, listing_df.lat))
listing_geo.crs = {'init': 'EPSG:4269'}

# Quebec census data
census_subdivisions = gpd.read_file('/Users/mathewzaharopoulos/Downloads/gcsd000b11a_e/gcsd000b11a_e.shp')
quebec_census = census_subdivisions[census_subdivisions.PRNAME== 'Quebec / Québec']

# Quebec flood zones
flood_zones = gpd.read_file('/Users/mathewzaharopoulos/Downloads/ZIS_Annexe2_2019_modifiee.gpkg')

# Convert CRS
flood_zones = flood_zones.to_crs(epsg=4269)
quebec_census = quebec_census.to_crs(epsg=4269)

# save flood zone json
flood_zones.geometry.to_file('flood_zones.json', driver='GeoJSON')

# Join listings and census zoning
listing_geo = gpd.sjoin(listing_geo, quebec_census)

## Select price range
# geo_range = listing_geo[geo_listing.ask_price <=1000000].copy()

#assign grouping name for statistics canada
g_name = 'CSDNAME'
# Drop regions with less than 30 listings
group_reg = listing_geo.groupby(g_name)
n_sig = group_reg['_id'].size() >= 10
n_sig = n_sig[n_sig == True]

r_sig = group_reg['type'].value_counts() >=10
r_sig = pd.DataFrame(r_sig)
r_sig.columns = ['bool']
r_sig = r_sig[r_sig['bool']== True]
r_sig = r_sig.reset_index()

# Create new df
POI_listings = listing_geo[listing_geo.CSDNAME.isin(list(n_sig.index))].copy()

# match regions and types with significant size
def filt(row):
    for r in r_sig.values[:,1::-1]:
        v_row = row[['type', g_name]].values

        if v_row[0] == r[0] and v_row[1] == r[1]:
            return row
# Filter to significant results
POI_listings = POI_listings.apply(filt, axis=1)

#copy data
house = POI_listings[POI_listings['type']== prop].copy()
#create detached, attached columns

#filter to detached homes
if prop == 'House':
    house['style'] = house.apply(lambda x: list(x['Building style'].keys())[0], axis=1)
    house = house[house['style'] == 'Detached']
    #wrangle data
    house['bathrooms'] = house['bathrooms'].apply(pd.to_numeric)

#price descriptive price statistic
group = house.groupby(g_name)

mean = group.ask_price.mean()
std = group.ask_price.std()
median = group.ask_price.median()
count = group.ask_price.count()

#create variables

avg_lot = round(group['Lot area'].mean(), 0)
avg_bathrooms = round(group['bathrooms'].mean(), 0)
avg_age = round(group['age'].mean(), 0)


#function to create values
transformer = lambda x,y,n: x.apply(pd.to_numeric) - y[n[g_name].values].values
transform_lot = lambda x: transformer(x['Lot area'], avg_lot, x)
transform_bathrooms = lambda x: transformer(x['bathrooms'], avg_bathrooms, x)
transform_age = lambda x: transformer(x['age'], avg_bathrooms, x)

#tranform variables
lot_diff_avg = (group.apply(transform_lot))
bathrooms_diff_avg = group.apply(transform_bathrooms)
age_diff_avg = group.apply(transform_age)
#create a dataframe of features

if prop == 'House':
    diffs = lot_diff_avg.to_frame().join(bathrooms_diff_avg).join(age_diff_avg)

else:
#     diffs = lot_diff_avg.join(bathrooms_diff_avg, rsuffix='_diff').join(age_diff_avg, rsuffix='_diff1')
    diffs = lot_diff_avg.to_frame().join(bathrooms_diff_avg).join(age_diff_avg)
#normalize index for join
# print(diffs.head())
diffs = diffs.reset_index().set_index('level_1').rename_axis(None).drop(columns=[g_name])

house = house.join(diffs, rsuffix='_diff')
print(f"Number of {prop}: {len(house)}")

#extract names
market_df['name'] = market_df['name'].apply(lambda x: x.split(',')[0])

#extract median price change
if prop == 'House':
    market_df['Single-family'] = market_df['Single-family'].apply(lambda x: x['quarter']['median_price']['percent'] if isinstance(x['quarter']['median_price']['percent'], int) else np.nan)
    market_df = market_df.dropna(subset=['Single-family'], axis=0).rename(columns={'name': g_name}).set_index(g_name)

    #create new dataframe
    value_change = quebec_census.set_index(g_name).join(market_df['Single-family'])
    value_change = value_change.dropna(subset=['Single-family'])

    #drop bottom quantile for choropleth resolution
    """
    bottom_q = value_change['Single-family'].quantile(0.15)
    query = value_change['Single-family'] > bottom_q
    value_change = value_change[query][::2]
    print(f"Market median price change minimum: {bottom_q}")
    """
    #minimum market change value

    query = value_change['Single-family'] > 0
    value_change = value_change[query].drop_duplicates()  #[::2]
    print(f"Market median price change minimum: 0")

#     value_change = value_change.drop_duplicates() 
# print(market_df.head())

Number of listings: 3372
Number of market regions: 1402


/Users/mathewzaharopoulos/Library/Python/3.7/lib/python/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/mathewzaharopoulos/Library/Python/3.7/lib/python/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(+init=epsg:4269 +type=crs != epsg:4269)
  "(%s != %s)" % (left_df.crs, right_df.crs)


Number of House: 755
Market median price change minimum: 0


In [328]:
#Market position selector for deals

metric_roi = lambda x,y: x > y
metric_market = lambda x,y: x - y

metric = metric_roi

if prop == 'House':
    m_var = 'Single-family'

def color_gen(row):
    # Evaluates the market comparables by economic region
    avg = mean.loc[row[g_name]]
    st = std.loc[row[g_name]]
    med = median.loc[row[g_name]]
    corrected_skew = (avg - med)
    if row.ask_price > (avg + st):
        return 'red'
    # Calculate the minimum distance from average price by standard deviation (normalize distribution)
    if gate:
        return 'green'
    else:
        return 'lightgray'

m_1 = folium.Map(location=[45.5, -73.583333], tiles='cartodbpositron', zoom_start=8)

mc = MarkerCluster(name='Total market')

mc3 = MarkerCluster(name='Deals', show=False)

for idx, row in house.iterrows():
    # Create variables
    
    r_pct = round(-100 + (mean.loc[row[g_name]]/row.ask_price * 100),2)
    dollar_return = round(round((r_pct/100), 2)* row.ask_price, 2)
    avg = mean.loc[row[g_name]]
    st = std.loc[row[g_name]]
    med = median.loc[row[g_name]]
    # Create gate including risk
    gate = (metric(r_pct, 50) 
            and row['Lot area_diff'] > - 250 or row['Lot area_diff'] == None 
            and row[g_name] in value_change[value_change[m_var] > 1].index and (avg - med) < 1000)

    #Build interface

    html = f"""
             <b>mls</b>: {int(row.list_num)} <br>
             <b>division</b> {row[g_name]}<br>
             <b>month/day</b>: {row['last_modified'].day} {row['last_modified'].month_name()}<br>
             <b>age</b>: {row.age}<br>
             <b>built</b>: {row['Year built']}<br>
             <b>lot</b>: {row['Lot area']}<br>
             <b>ask</b>: ${row.ask_price},  <br>
             <b>avg</b>: ${round(avg, 2)},  <br>
            <b>median</b>: ${round(median.loc[row[g_name]], 2)},  <br>
             <b>std</b>: ${round(st,2)} <br> <br>
             <b>additional</b>: {row['Additional features']}<br><br>
             <b>description</b>: {row.description}<br><br>
             <a href='https://www.centris.ca/en/' target='_blank' rel='noopener noreferrer'>Open a Centris Tab! Copy the MLS number.</a>"""
    iframe = folium.IFrame(html=html, width=250, height=300)
    popup = folium.Popup(iframe, max_width=2650)
    mc.add_child(folium.Marker([row.lat, row.lng],icon=folium.Icon(color=color_gen(row), icon='dollar-sign'),
                                                               popup=popup,
                                             tooltip=(f"""
                                                 <b>mls</b>: {int(row.list_num)} <br>
                                                 <b>ROI %</b>: {round(-1 + r_pct, 2)}<br>
                                                 <b>ROI</b>: ${dollar_return}<br>
                                                 <b>lot compare</b>: {row['Lot area_diff']}<br>
                                                 <b>age compare</b>: {row['age_diff']} <br>
                                                 <b>bathrooms compare</b>: {row['bathrooms_diff']}<br>""")))
    
    

    

    


    if gate:
        popup2 = folium.Popup(iframe, max_width=2650)
        mc3.add_child(folium.Marker([row.lat, row.lng],icon=folium.Icon(color=color_gen(row), icon='dollar-sign', prefix='fa'),
                                                           popup=popup2,
                                               tooltip=(f"""
                                                 <b>mls</b>: {int(row.list_num)} <br>
                                                 <b>ROI %</b>: {round(-1 + r_pct, 2)}<br>
                                                 <b>ROI</b>: ${dollar_return}<br>
                                                 <b>lot compare</b>: {row['Lot area_diff']}<br>
                                                 <b>age compare</b>: {row['age_diff']} <br>
                                                 <b>bathrooms compare</b>: {row['bathrooms_diff']}<br>""")))
        


folium.Choropleth(geo_data=value_change.__geo_interface__,
                 data=value_change[m_var],
                 key_on='feature.id',
                 fill_color='YlGnBu',
                 name='Market price change').add_to(m_1)

folium.GeoJson('flood_zones.json', name='Flood zones', show=False).add_to(m_1)
m_1.add_child(MeasureControl())


m_1.add_child(mc3)

m_1.add_child(mc)


folium.LayerControl().add_to(m_1)
# m_1
m_1.save('may_19_centris_tooltip.html')

In [99]:
m_1.save('may_18_centris.html')

Index(['Beloeil', 'Blainville', 'Boisbriand', 'Boucherville', 'Brossard',
       'Candiac', 'Cantley', 'Carignan', 'Chambly', 'Châteauguay',
       'Deux-Montagnes', 'Dollard-Des Ormeaux', 'Drummondville',
       'L'Assomption', 'La Prairie', 'La Pêche', 'Lavaltrie', 'Magog',
       'Mascouche', 'Mercier', 'Mirabel', 'Mont-Royal', 'Mont-Saint-Hilaire',
       'Otterburn Park', 'Pincourt', 'Prévost', 'Rosemère', 'Saint-Amable',
       'Saint-Bruno-de-Montarville', 'Saint-Calixte', 'Saint-Constant',
       'Saint-Eustache', 'Saint-Hyacinthe', 'Saint-Jérôme', 'Saint-Lazare',
       'Saint-Sauveur', 'Saint-Zotique', 'Sainte-Anne-des-Plaines',
       'Sainte-Julie', 'Sainte-Julienne', 'Sainte-Marthe-sur-le-Lac',
       'Salaberry-de-Valleyfield', 'Sorel-Tracy', 'Trois-Rivières',
       'Val-des-Monts', 'Varennes', 'Vaudreuil-Dorion', 'Westmount'],
      dtype='object', name='CSDNAME')

In [300]:
value_change.loc['Sainte-Julienne'].list_num.to_clipboard(index=False,header=False)

AttributeError: 'Series' object has no attribute 'list_num'

In [310]:
listing_df.iloc[0].list_num.astype(str).to_clipboard(index=False)

AttributeError: 'numpy.str_' object has no attribute 'to_clipboard'

In [318]:
value_change.loc['Beaconsfield']

CSDUID                                                     2466107
CSDTYPE                                                          V
PRUID                                                           24
PRNAME                                             Quebec / Québec
CDUID                                                         2466
CDNAME                                                    Montréal
CDTYPE                                                          TÉ
CMAUID                                                         462
CMANAME                                                   Montréal
CMATYPE                                                          B
CMAPUID                                                      24462
SACTYPE                                                          1
SACCODE                                                        462
ERUID                                                         2440
ERNAME                                                    Mont

ValueError: Lengths must match to compare